<a href="https://colab.research.google.com/github/GuilleLondero/ML_challenge_one_DS_telecomX/blob/main/Challenge_TelecomX2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importaciones**

In [9]:
# Conexion de google colab con drive:
from google.colab import drive

# Importar librerías básicas
import pandas as pd
import numpy as np

# **Extracción del Archivo Tratado**

In [10]:

df_ml_inicial = pd.read_csv('/content/drive/MyDrive/ONE/DATA SCIENCE/ML_challenge_TelecomX2/telecomX1_datos_tratados.csv')

print("✅ Datos cargados desde Drive")
print(f"Shape: {df_ml_inicial.shape}")

✅ Datos cargados desde Drive
Shape: (7043, 22)


In [11]:
df_ml_inicial.head()

,Id_Cliente,Fuga_Cliente,Genero,Ciudadano_Mayor,Tiene_Pareja,Tiene_Dependientes,Antiguedad_Meses,Servicio_Telefono,Lineas_Multiples,Servicio_Internet,...,Proteccion_Dispositivo,Soporte_Tecnico,TV_Streaming,Peliculas_Streaming,Tipo_Contrato,Facturacion_Digital,Metodo_Pago,Cargo_Mensual,Cargo_Total,Cargo_Diario
0,0002-ORFBO,0,Female,0,1,1,9,1,0,DSL,...,0,1,1,0,One year,1,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,0,Male,0,0,0,9,1,1,DSL,...,0,0,0,1,Month-to-month,0,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,1,Male,0,0,0,4,1,0,Fiber optic,...,1,0,0,0,Month-to-month,1,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,1,Male,1,1,0,13,1,0,Fiber optic,...,1,0,1,1,Month-to-month,1,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,1,Female,1,1,0,3,1,0,Fiber optic,...,0,1,1,0,Month-to-month,1,Mailed check,83.9,267.40,2.80


# **Eliminación de Columnas Irrelevantes**